In [10]:
import json
import pandas as pd

* Read the results of the optuna executions on the complete input

In [11]:
with open("ga_results_optuna.json", "r") as f:
    data = json.load(f)

* Compute averages for each instance

In [12]:
def extract_instance_type(instance_name):
    # extracts the prefix up to ORD_10 (or similar)
    parts = instance_name.split('-')
    for i, part in enumerate(parts):
        if part.startswith('Ord') or part.startswith('ORD') or part.startswith('Ord_'):
            return '-'.join(parts[:i+1])

    return '_'.join(instance_name.split('_')[:2])

records = []
best_fitness_per_type = {}

for instance, content in data.items():
    try:
        detailed_results = content['method:']['GA']['swap']['detailed_trial_results']
    except KeyError:
        continue
    # find max fitness in detailed_trial_results
    max_fitness = max(
        trial['trial_results']['trial-0']['fitness']
        for trial in detailed_results
    )
    instance_type = extract_instance_type(instance)
    records.append({
        'instance': instance,
        'instance_type': instance_type,
        'fitness': max_fitness
    })

df = pd.DataFrame(records).set_index('instance')

# compute best fitness per instance_type
best_fitness_per_type = df.groupby('instance_type')['fitness'].max()

# compute RPD for each instance
def compute_rpd(row):
    best = 1 / best_fitness_per_type[row['instance_type']]
    return ((1 / row['fitness']) - best) / best

df['RPD'] = df.apply(compute_rpd, axis=1)

* Present the fitness and RPD averages per instance

In [13]:
df[['fitness', 'RPD']].sort_index()

,fitness,RPD
instance,,
S_3-Ord_10-I_0-WIP_0,0.015152,0.000000
S_3-Ord_10-I_1-WIP_0,0.011494,0.318182
S_3-Ord_10-I_2-WIP_0,0.009346,0.621212
S_3-Ord_30-I_0-WIP_0,0.002000,0.136364
S_3-Ord_30-I_1-WIP_0,0.002037,0.115909
S_3-Ord_30-I_2-WIP_0,0.002273,0.000000
S_3-Ord_50-I_0-WIP_0,0.003509,0.000000
S_3-Ord_50-I_1-WIP_0,0.003497,0.003509
S_3-Ord_50-I_2-WIP_0,0.003509,0.000000


* Group and present the mean results per stages-orders categories

In [14]:
df.groupby('instance_type')[["fitness", "RPD"]].mean()

,fitness,RPD
instance_type,,
S_3-Ord_10,0.011997,0.313131
S_3-Ord_30,0.002103,0.084091
S_3-Ord_50,0.003505,0.001170
S_6-Ord_10,0.008303,0.371134
S_6-Ord_30,0.003669,0.334855
S_6-Ord_50,0.001845,0.570931
S_9-Ord_10,0.005551,0.036398
S_9-Ord_30,0.002638,0.390046
S_9-Ord_50,0.002042,0.376091


* Compute averages per orders and per stages

In [15]:
# average per orders
df['Ord_type'] = df['instance_type'].str.extract(r'(Ord_\d+)')
ord_averages = df.groupby('Ord_type')[['fitness', 'RPD']].mean()

# average per stages
df['S_type'] = df['instance_type'].str.extract(r'(S_\d+)')
s_averages = df.groupby('S_type')[['fitness', 'RPD']].mean()

* Present order averages

In [16]:
ord_averages

,fitness,RPD
Ord_type,,
Ord_10,0.008617,0.240221
Ord_30,0.002803,0.269664
Ord_50,0.002464,0.316064


* Present stage averages

In [17]:
s_averages

,fitness,RPD
S_type,,
S_3,0.005868,0.132797
S_6,0.004606,0.425640
S_9,0.003411,0.267512
